In [ ]:
import os
import platform
from datetime import datetime

import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.python.keras import layers
from tensorflow.python.keras.metrics import Precision, Recall, BinaryAccuracy

from modules.dataset_utils import DatasetUtils

#######################################################################################################################
# PARÁMETROS DEFINIDOS EN FUNCIÓN DEL USUARIO
#######################################################################################################################

# Ruta hacia los CSV del dataset
train_file_relative = './data/700Train.csv'
validation_file_relative = './data/700Validation.csv'
test_file_relative = './data/700Test.csv'

# Ruta al directorio de imágenes con máscara aplicada
masked_dir_relative = '../local/output_edited'

model_dir_relative = "./models"

# Ruta relativa hacia la librería ai4eutils
ai4eutils_relative = "./repos/ai4eutils"

# Ruta relativa hacia la librería CameraTraps
CameraTraps_relative = "./repos/CameraTraps"

#######################################################################################################################
# PARÁMETROS PARA CNN
#######################################################################################################################

IMG_HEIGHT = IMG_WEIGHT = 500
EPOCHS = 50
BATCH_SIZE = 16

#######################################################################################################################
#######################################################################################################################

home = os.path.expanduser("~")

ai4utils = os.path.abspath(ai4eutils_relative)
CameraTraps = os.path.abspath(CameraTraps_relative)
train_file = os.path.abspath(train_file_relative)
validation_file = os.path.abspath(validation_file_relative)
test_file = os.path.abspath(test_file_relative)
masked_dir = os.path.abspath(masked_dir_relative)
model_dir = os.path.abspath(model_dir_relative)

try:
    os.environ['PYTHONPATH']
except KeyError:
    os.environ['PYTHONPATH'] = ""
if platform.system() == 'Windows':
    os.environ['PYTHONPATH'] += (";" + ai4utils)
    os.environ['PYTHONPATH'] += (";" + CameraTraps)
else:
    os.environ['PYTHONPATH'] += (":" + ai4utils)
    os.environ['PYTHONPATH'] += (":" + CameraTraps)

print(
    '==================================================================================================================================')
print('PYTHONPATH: ' + os.environ['PYTHONPATH'])
print('')
print('ai4eutils PATH: ' + '\t' + ai4utils)
print('CameraTraps PATH: ' + '\t' + CameraTraps)
print('Train CSV PATH: ' + '\t' + train_file)
print('Validation CSV PATH: ' + '\t' + validation_file)
print('Test CSV PATH: ' + '\t' + '\t' + test_file)
print('masked PATH: ' + '\t' + '\t' + masked_dir)
print('models PATH: ' + '\t' + '\t' + model_dir)
print(
    '==================================================================================================================================')

In [ ]:
file_path, labels = DatasetUtils.load_dataset(train_file, masked_dir)

TRAIN_BUFFER = len(labels)
train_dataset = tf.data.Dataset.from_tensor_slices((file_path, labels))
train_dataset = train_dataset.map(DatasetUtils.load_image).map(DatasetUtils.normalize_images).map(
    DatasetUtils.resize_image).shuffle(TRAIN_BUFFER).batch(BATCH_SIZE)

file_path, labels = DatasetUtils.load_dataset(validation_file, masked_dir)
VALIDATION_BUFFER = len(labels)
validation_dataset = tf.data.Dataset.from_tensor_slices((file_path, labels))
validation_dataset = validation_dataset.map(DatasetUtils.load_image).map(DatasetUtils.normalize_images).map(
    DatasetUtils.resize_image).shuffle(VALIDATION_BUFFER).batch(BATCH_SIZE)

file_path, labels = DatasetUtils.load_dataset(test_file, masked_dir)
TEST_BUFFER = len(labels)
test_dataset = tf.data.Dataset.from_tensor_slices((file_path, labels))
test_dataset = test_dataset.map(DatasetUtils.load_image).map(DatasetUtils.normalize_images).map(
    DatasetUtils.resize_image).shuffle(TEST_BUFFER).batch(BATCH_SIZE)

In [ ]:
model = keras.Sequential([

    layers.Conv2D(16, kernel_size=3, padding='same', activation=tf.nn.relu, input_shape=(IMG_HEIGHT, IMG_WEIGHT, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(32, kernel_size=3, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, kernel_size=3, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(32, kernel_size=3, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(16, kernel_size=3, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D(2, 2),

    # Full connected
    layers.Flatten(),
    layers.Dense(1024, kernel_regularizer=keras.regularizers.l2(1e-3), activation=tf.nn.relu),
    layers.Dropout(0.5),
    layers.Dense(1024, kernel_regularizer=keras.regularizers.l2(1e-3), activation=tf.nn.relu),
    layers.Dropout(0.5),

    # Capa de salida
    layers.Dense(1, activation=tf.nn.sigmoid)
])

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4), loss=[keras.losses.BinaryCrossentropy()],
              metrics=['accuracy'])
model.summary()

In [ ]:
hist = model.fit(train_dataset, epochs=EPOCHS, validation_data=validation_dataset, batch_size=BATCH_SIZE)

In [ ]:
ax = pd.DataFrame(data=hist.history).plot(figsize=(15, 7))
ax.grid()
_ = ax.set(title="Training loss and accuracy", xlabel="Epochs")
_ = ax.legend(["Training loss", "Training accuracy", "Validation loss", "Validation accuracy"])

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='blue', label='loss', )
plt.plot(hist.history['val_loss'], color='green', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='orange', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='red', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
results = model.evaluate(test_dataset, verbose=1)

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()
for sample in test_dataset:
    X = sample[0]
    y = sample[1]
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)
print('Precision: ' + str(pre.result()))
print('Recall: ' + str(re.result()))
print('BinaryAccuracy: ' + str(acc.result()))

In [ ]:
date = datetime.utcnow().strftime('%Y-%m-%d_%H-%M')
if platform.system() == 'Windows':
    path = model_dir + '\\' + date
else:
    path = model_dir + '/' + date

model.save(path)